## 1. 웹스크래핑 연습문제

* user-agent 요청헤더를 반드시 설정해야 한다.

In [ ]:
# requests 라이브러리 설치여부 확인
!pip show requests
# beautifulsoup4 라이브러리 설치여부 확인
!pip show beautifulsoup4

### 질문 1: Daum 뉴스기사 제목 스크래핑하기

In [ ]:
import requests
from bs4 import BeautifulSoup

# 다음 경제 뉴스 URL
url = 'https://news.daum.net/economy'
print(url)

# 요청 헤더 설정 : 브라우저 정보
req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}

# requests 의 get() 함수 호출하기 
res = requests.get(url, headers=req_header)
print(type(res))
print(res.status_code)
# 응답(response)이 OK 이면 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성
if res.ok :
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')
    # CSS 선택자를 사용해서 a tag 목록 가져오기
    a_tags = soup.select("ul.list_newsheadline2 a[href*='https://v.daum.net/v/']")
    print(type(a_tags), len(a_tags))
    # <a> 태그 리스트 순회하기    
    for a_tag in a_tags :
        link = a_tag['href']
        strong_tag = a_tag.select_one('div.cont_thumb strong.tit_txt')
        title = strong_tag.text
        
        print(link, title)  
else :
    # 응답(response)이 Error 이면 status code 출력    
    print(f'Error Code = {res.status_code}')

### 질문2:  여러개의 section 중 하나를 선택해서 url에서 뉴스기사의 링크와 제목을 출력하는 코드를 함수로 작성하기

In [ ]:
import requests
from bs4 import BeautifulSoup

section_dict = {'기후/환경':'climate','사회':'society','경제':'economy','정치':'politics',\
             '국제':'world','문화':'culture','생활':'life','IT/과학':'tech','인물':'people'}

def print_news(section_name):  #print_new(life,'생활') 
    sid = section_dict.get(section_name)
    url = f'https://news.daum.net/{sid}'
    print(f'======> {url} {section_name} 뉴스 <======')
    req_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
}
    res = requests.get(url, headers=req_header)
    if res.ok :
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        # CSS 선택자를 사용해서 a tag 목록 가져오기
        a_tags = soup.select("ul.list_newsheadline2 a[href*='https://v.daum.net/v/']")
        # <a> 태그 리스트 순회하기    
        for a_tag in a_tags :
            link = a_tag['href']
            strong_tag = a_tag.select_one('div.cont_thumb strong.tit_txt')
            title = strong_tag.text
            
            print(link)
            print(title)
    else :
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')

print_news('경제')

### 2-1. Nate 뉴스기사 제목 스크래핑하기

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

section_dict = { '최신 뉴스': 'n0100', '정치': 'n0200', '경제': 'n0300', '사회': 'n0400', '세계': 'n0500', 'IT/과학': 'n0600'
}

def nate_news_scrape(section_name):
    sid = section_dict.get(section_name)
    url = f'https://news.nate.com/recent?mid={sid}'
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
    }
    
    res = requests.get(url, headers=req_header)
    
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        # 뉴스 기사 div 태그 선택 (각 뉴스 기사 단위)
        articles = soup.select('div.mduSubjectList div.mlt01')

        for article in articles[:5]:
            # img 엘리먼트 존재 여부 체크
            img_tag = article.select_one('img')
            if img_tag and img_tag.get('src'):
                img_url = urljoin(url, img_tag['src'])
                display(Image(url=img_url))  # 이미지 출력
            else:
                print('이미지 없음')

            # 기사 제목 + 기사 링크
            a_tag = article.select_one('a')
            title_tag = a_tag.select_one('h2')
            title = title_tag.text.strip()
            link = urljoin(url, a_tag['href'])
            print('기사제목:', title)
            print('기사링크:', link)

    else:
        # 응답(response)이 Error 이면 status code 출력  
        print(f'Error Code = {res.status_code}')

nate_news_scrape('세계')

### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

def download_one_episode(title, no, url):
    # 기본 설정
    req_header = {'referer': url}
    imgdir_name = f'img/{title}/{no}'

    # 이미지 저장 폴더가 없으면 생성
    os.makedirs(imgdir_name, exist_ok=True)

    # 웹 페이지 요청 및 확인
    res = requests.get(url)
    if not res.ok:
        print(f'Error Code = {res.status_code}')
        exit()

    # 이미지 URL 추출
    soup = BeautifulSoup(res.text, 'html.parser')
    img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

    # 이미지 다운로드
    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:
            img_data = res.content
            file_path = os.path.join(imgdir_name, os.path.basename(img_url))
            with open(file_path, 'wb') as file:
                print(f'Writing to {file_path} ({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error Code = {res.status_code} for {img_url}')

download_one_episode('입학용병',271,'https://comic.naver.com/webtoon/detail?titleId=758150&no=272&week=sun')

### 2-3. 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

def download_all_episode(title, episode_url):
    
    req_header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36'
    }
    
    res = requests.get(url, headers=req_header)
    
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        # 뉴스 기사 div 태그 선택 (각 뉴스 기사 단위)
        articles = soup.select('div.mduSubjectList div.mlt01')

        for article in articles[:5]:
            # img 엘리먼트 존재 여부 체크
            img_tag = article.select_one('img')
            if img_tag and img_tag.get('src'):
                img_url = urljoin(url, img_tag['src'])
                display(Image(url=img_url))  # 이미지 출력
            else:
                print('이미지 없음')

            # 기사 제목 + 기사 링크
            a_tag = article.select_one('a')
            title_tag = a_tag.select_one('h2')
            title = title_tag.text.strip()
            link = urljoin(url, a_tag['href'])
            print('기사제목:', title)
            print('기사링크:', link)

    else:
        # 응답(response)이 Error 이면 status code 출력  
        print(f'Error Code = {res.status_code}')

def download_one_episode(title, no, url):
    # 기본 설정
    req_header = {'referer': url}
    imgdir_name = f'img/{title}/{no}'

    # 이미지 저장 폴더가 없으면 생성
    os.makedirs(imgdir_name, exist_ok=True)

    # 웹 페이지 요청 및 확인
    res = requests.get(url)
    if not res.ok:
        print(f'Error Code = {res.status_code}')
        exit()

    # 이미지 URL 추출
    soup = BeautifulSoup(res.text, 'html.parser')
    img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

    # 이미지 다운로드
    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:
            img_data = res.content
            file_path = os.path.join(imgdir_name, os.path.basename(img_url))
            with open(file_path, 'wb') as file:
                print(f'Writing to {file_path} ({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'Error Code = {res.status_code} for {img_url}')

200


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [ ]:
# 캡차 키 발급 요청


In [ ]:
# 캡차 이미지 요청


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [ ]:
# 사용자 입력값 검증 요청



Error Code: 403


In [ ]:
import requests
import pprint

headers = {
    'X-Naver-Client-Id': '',
    'X-Naver-Client-Secret': '',
}

payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json'


# requests get(url, params, headers) 요청 

# json() 함수로 응답 결과 가져오오기
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'

# 'data/nhnblog.txt' 파일 생성하기
